In [158]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

import json
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool

from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph,END
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

load_dotenv()

True

In [159]:
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [160]:
# model = ChatGroq(temperature=0, model="llama3-70b-8192", streaming=False)
# model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)
model = ChatAnthropic(model="claude-3-haiku-20240307")

In [161]:
model.invoke('hola')

AIMessage(content='¡Hola! ¿Cómo estás?', response_metadata={'id': 'msg_01JQjUQ5MQFbkzHpd6daNb5g', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 9, 'output_tokens': 20}}, id='run-7fd387d0-839d-4036-b4b4-a7a46bbd5713-0')

In [162]:
@tool
def multiply(first_number: int, second_number: int):
    """Multiplica dos números."""
    return first_number * second_number



In [163]:
# model_with_tools = model.bind(tools=[convert_to_openai_tool(multiply)])
# response = model_with_tools.invoke('Cuanto es 35 * 46?')
# response

In [164]:
tools = [multiply]
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke('Cuanto es 35 * 46?')
response

AIMessage(content=[{'text': 'Vamos a calcular 35 * 46 utilizando la herramienta "multiply":', 'type': 'text'}, {'id': 'toolu_01SoxQtkkSFFdTLuHM7oDFnG', 'input': {'first_number': 35, 'second_number': 46}, 'name': 'multiply', 'type': 'tool_use'}], response_metadata={'id': 'msg_012TtBmacBrqpP4npuWC6RuT', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 374, 'output_tokens': 96}}, id='run-67fe3aae-5f93-411b-a463-95c0f9519d16-0', tool_calls=[{'name': 'multiply', 'args': {'first_number': 35, 'second_number': 46}, 'id': 'toolu_01SoxQtkkSFFdTLuHM7oDFnG'}])

In [165]:
# tools = [multiply]
# model_with_tools = model.bind_tools(tools)
# response = model_with_tools.invoke('Que es un LLM?')
# response

In [166]:
tool_calls = response.tool_calls
tool_calls

[{'name': 'multiply',
  'args': {'first_number': 35, 'second_number': 46},
  'id': 'toolu_01SoxQtkkSFFdTLuHM7oDFnG'}]

In [167]:
tool_calls

[{'name': 'multiply',
  'args': {'first_number': 35, 'second_number': 46},
  'id': 'toolu_01SoxQtkkSFFdTLuHM7oDFnG'}]

In [168]:
# tool_calls = response.additional_kwargs.get('tool_calls')
# tool_calls

In [169]:
type(tool_calls)

list

In [170]:
for tool_call in tool_calls:
    print('Function Name:',tool_call.get('name'))
    print('Function Arguments:',tool_call.get('args'))
    print(tool_call)

Function Name: multiply
Function Arguments: {'first_number': 35, 'second_number': 46}
{'name': 'multiply', 'args': {'first_number': 35, 'second_number': 46}, 'id': 'toolu_01SoxQtkkSFFdTLuHM7oDFnG'}


### Construcción del Gráfico con llamada condicional a aristas y herramientas

In [171]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    

In [172]:
def invoke_model(state):
    messages = state['messages']
    question = messages[-1]## Fetching the user question
    return {"messages":[model_with_tools.invoke(question)]}

In [173]:
def invoke_tool(state):
    print("******************DERIvADO***************")
    tool_calls = state['messages'][-1].additional_kwargs.get("tool_calls", [])
    multiply_call = None

    for tool_call in tool_calls:
        if tool_call.get("function").get("name") == "multiply":
            multiply_call = tool_call

    if multiply_call is None:
        raise Exception("No se ha encontrado ninguna entrada para sumar.")

    res = multiply.invoke(
        json.loads(multiply_call.get("function").get("arguments"))
        # Convierte una cadena JSON a un diccionario de Python '{"first_number": 35, "second_number": 46}'
    )

    return {"messages" : [res]
    }

In [174]:
def router(state):
    tool_calls = state['messages'][-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "multiply"
    else:
        return "end"

In [175]:
graph = StateGraph(AgentState)
graph.add_node("agent", invoke_model)
graph.add_node("tool", invoke_tool)
graph.add_edge("tool", END)
graph.add_conditional_edges("agent", router, {
    "multiply": "tool",
    "end": END,
})
graph.set_entry_point("agent")
app = graph.compile()

In [176]:
output = app.invoke({"messages": ["Cuanto es 123 * 456?"]})

In [177]:
output

{'messages': ['Cuanto es 123 * 456?',
  AIMessage(content=[{'id': 'toolu_01Vw1dKDnnL5zHy8N28UaXc5', 'input': {'first_number': 123, 'second_number': 456}, 'name': 'multiply', 'type': 'tool_use'}], response_metadata={'id': 'msg_01NK1SxPLJmVtQKou9DqmD3i', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 374, 'output_tokens': 72}}, id='run-475bf1e4-bcb8-48d0-8e46-b7452b820845-0', tool_calls=[{'name': 'multiply', 'args': {'first_number': 123, 'second_number': 456}, 'id': 'toolu_01Vw1dKDnnL5zHy8N28UaXc5'}])]}

In [178]:
output['messages'][-1]

AIMessage(content=[{'id': 'toolu_01Vw1dKDnnL5zHy8N28UaXc5', 'input': {'first_number': 123, 'second_number': 456}, 'name': 'multiply', 'type': 'tool_use'}], response_metadata={'id': 'msg_01NK1SxPLJmVtQKou9DqmD3i', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 374, 'output_tokens': 72}}, id='run-475bf1e4-bcb8-48d0-8e46-b7452b820845-0', tool_calls=[{'name': 'multiply', 'args': {'first_number': 123, 'second_number': 456}, 'id': 'toolu_01Vw1dKDnnL5zHy8N28UaXc5'}])

In [179]:
output = app.invoke({"messages": ["Que es LLM?"]})

In [180]:
output

{'messages': ['Que es LLM?',
  AIMessage(content='LLM significa "Large Language Model" (Modelo de Lenguaje a Gran Escala). Se trata de un tipo de modelo de aprendizaje automático basado en redes neuronales profundas que ha sido entrenado en grandes cantidades de datos de texto para aprender a comprender y generar lenguaje natural.\n\nAlgunas características clave de los LLM:\n\n- Están entrenados en enormes cantidades de datos de texto de Internet, libros, artículos, etc. Lo que les permite aprender sobre una gran variedad de temas y dominios.\n\n- Pueden comprender y generar texto de manera muy natural, parecido a como lo haría un ser humano.\n\n- Tienen la capacidad de realizar una amplia gama de tareas relacionadas con el lenguaje, como responder preguntas, hacer resúmenes, traducir entre idiomas, generar texto creativo, etc.\n\n- Utilizan arquitecturas como Transformers que les permiten aprender representaciones semánticas profundas del lenguaje.\n\n- Ejemplos prominentes de LLM in

In [181]:
print(output['messages'][-1])

content='LLM significa "Large Language Model" (Modelo de Lenguaje a Gran Escala). Se trata de un tipo de modelo de aprendizaje automático basado en redes neuronales profundas que ha sido entrenado en grandes cantidades de datos de texto para aprender a comprender y generar lenguaje natural.\n\nAlgunas características clave de los LLM:\n\n- Están entrenados en enormes cantidades de datos de texto de Internet, libros, artículos, etc. Lo que les permite aprender sobre una gran variedad de temas y dominios.\n\n- Pueden comprender y generar texto de manera muy natural, parecido a como lo haría un ser humano.\n\n- Tienen la capacidad de realizar una amplia gama de tareas relacionadas con el lenguaje, como responder preguntas, hacer resúmenes, traducir entre idiomas, generar texto creativo, etc.\n\n- Utilizan arquitecturas como Transformers que les permiten aprender representaciones semánticas profundas del lenguaje.\n\n- Ejemplos prominentes de LLM incluyen GPT-3, BERT, T5, entre otros desar